# Data and Packages

In [1]:
# Load packages
import numpy as np
import scipy as sp

In [2]:
# Data
s = np.array([0.15,0.15,0.3,0.3]) # Market shares, 0.1 for outside good
m = 0.5 # m = (p-c)/p  => c = p(1-m) # price-cost margin for 1st firm (for calibration)
p = np.array([1,1,1,1]) # pre-merger prices
p2f = np.array([1,2,3,4]) # product to firm mapping
P = 1 # market price
Q = 1 # market quantity

# Functions

In [3]:
def ownershipMatrix(p2f):
    '''Converts a Jx1 vector mapping product-to-firms into JxJ Ownership matrix Ω
        Ω_{i,j} = 1 if the same firm produces product i and j, else 0. '''
    J = len(p2f) # Number of products
    F = len(np.unique(p2f)) # Number of firms
    Ω = np.zeros((J,J)) # Ownership matrix
    
    for i in range(J):
        for j in range(J):
            if p2f[i] == p2f[j]: # firm producing product i is same firm that produces product j
                Ω[i,j] = 1
                
    #print('No. of products:',J)
    #print('No. of firms:',F)
    return Ω 
    
Ω = ownershipMatrix(p2f)

In [4]:
def calibrateLogit(m,s,p,p2f):
    '''Input: margin, shares (quantities),prices, product-to-firm mapping 
    Output: α: Calibrated price-coeff, ξ: mean non-price values, mc: marginal cost, 
    type_j: Nocke Shutz types, type: firm type, dqdp: demand derivatives, div: diversion matrix'''

    Ω = ownershipMatrix(p2f) 
    J = len(p2f) 
    c1 = p[0]*(1-m) # Cost of 1st firm

    # Generate Cross price derivatives
    temp = -np.outer(s,s) #tcrossprod
    np.fill_diagonal(temp, s*(1-s))

    # Calculate α from the demand
    if 1==len(p2f[p2f==1]):
        α = -1/(1-s[0])/(p[0]-c1)
    else:
        pass

    # Cross price derivatives
    dqdp = temp*α

    # Marginal costs
    c = p + np.dot(np.linalg.inv(Ω*dqdp.T),s)

    # Diversion Matrix D[k,j] = s_k / (1-sj) and -1 on diagonal
    div = np.multiply(s,1/(1-s).reshape(-1,1))
    np.fill_diagonal(div, -1)

    # Mean Valuations
    ξ = np.log(s/(1-np.sum(s)))-α*p

    # Type
    type_j = np.exp(ξ-α*c)
    Type = np.bincount(p2f-1, weights=type_j)
    
    return α, ξ, c, type_j, Type, dqdp, div

α, ξ, c, type_j, Type, dqdp, div = calibrateLogit(m,s,p,p2f)

print(α, ξ)
print(dqdp)
print(div)

-2.3529411764705883 [2.75840628 2.75840628 3.45155347 3.45155347]
[[-0.3         0.05294118  0.10588235  0.10588235]
 [ 0.05294118 -0.3         0.10588235  0.10588235]
 [ 0.10588235  0.10588235 -0.49411765  0.21176471]
 [ 0.10588235  0.10588235  0.21176471 -0.49411765]]
[[-1.          0.17647059  0.35294118  0.35294118]
 [ 0.17647059 -1.          0.35294118  0.35294118]
 [ 0.21428571  0.21428571 -1.          0.42857143]
 [ 0.21428571  0.21428571  0.42857143 -1.        ]]


In [5]:
def foc(p,c,s,dqdp,Ω):
    '''Checks the FOC conditions in the Bertrand-Nash price competition, for any generic demand derivatives. 
    Inputs: Vectors of prices, marginal costs, quantities (shares), demand derivative matrix, ownership (structure) matrix.
    Output: Upward Pricing Pressure, or the value of FOC for each product
    '''
    FOC = -p + c - np.dot(np.linalg.inv(Ω*dqdp.T),s)
    return np.round(FOC,4)

foc(p,c,s,dqdp,Ω)

array([0., 0., 0., 0.])

In [6]:
def quant_logit(p, α, ξ):
    '''Input: Vector of Prices p, price elasticity α and vector of unobserved quality ξ
    Output: Vector of shares (quantities) '''
    s = np.exp(p*α+ξ) * (1/(1+np.sum(np.exp(p*α+ξ))))
    return s

quant_logit(p, α, ξ)

array([0.15, 0.15, 0.3 , 0.3 ])

In [7]:
def dqdp_logit(p,α,ξ):
    '''Demand derivatives for Logit Demand
    Input: prices, price elasticity, unobserved quality
    Output: JxJ demand derivative matrix'''
    s = quant_logit(p, α, ξ)
    temp = -np.outer(s,s) #tcrossprod
    np.fill_diagonal(temp, s*(1-s))
    dqdp = α * temp
    return dqdp

dqdp_logit(p,α,ξ)

array([[-0.3       ,  0.05294118,  0.10588235,  0.10588235],
       [ 0.05294118, -0.3       ,  0.10588235,  0.10588235],
       [ 0.10588235,  0.10588235, -0.49411765,  0.21176471],
       [ 0.10588235,  0.10588235,  0.21176471, -0.49411765]])

In [8]:
foc(p,c,quant_logit(p, α, ξ),dqdp,Ω)

array([-0., -0., -0., -0.])

In [9]:
def foc_logit(p,c,α,ξ,p2f):
    '''Check FOC for Logit'''
    Ω = ownershipMatrix(p2f)
    dqdp = dqdp_logit(p,α,ξ)
    s = quant_logit(p,α,ξ)
    FOC = foc(p,c,s,dqdp,Ω)
    return FOC

foc_logit(p,c,α,ξ,p2f)

array([0., 0., 0., 0.])

# Merger Simulation 1: Firm1 with Firm 2

In [10]:
# Step 1: Calibrate to obtain Structural Parameters
α, ξ, c, type_j, Type, dqdp, div = calibrateLogit(m,s,p,p2f)

In [11]:
# Step 2: Check the FOC conditions
foc_logit(p,c,α,ξ,p2f)

array([0., 0., 0., 0.])

In [12]:
# Step 3: Create new ownership matrix relating to post-merger situation
p2f_new = np.array([1,1,3,4])

In [13]:
# Step 4: Calculate value of FOC with post-merger structure with pre-merger prices
# This is upward pricing pressure
foc_logit(p,c,α,ξ,p2f_new)

array([0.1071, 0.1071, 0.    , 0.    ])

In [14]:
# Step 5: Calculate the Post Merger Prices
p_new = sp.optimize.fsolve(foc_logit, x0=np.array([0,0,0,0]), args=(c,α,ξ,p2f_new))
print(p_new)

[1.07958482 1.07957795 1.0115344  1.01152829]


In [15]:
# Step6: Calculate new shares/quantities
q_new = quant_logit(p_new, α, ξ)
print(q_new)

[0.13335806 0.13336021 0.3130314  0.31303589]


# Merger Simulation 2: Reduced Costs

In [16]:
p_new = sp.optimize.fsolve(foc_logit, x0=np.array([0,0,0,0]), args=(c*np.array([0.9,0.9,1,1]),α,ξ,p2f_new))
print(p_new)

q_new = quant_logit(p_new, α, ξ)
print(q_new)

[1.04201582 1.04204045 1.00612626 1.00613118]
[0.14107725 0.14106908 0.3070163  0.30701274]


# Merger Simulation 3: Collusion

In [17]:
p_new = sp.optimize.fsolve(foc_logit, x0=np.array([1,2,1,1]), args=(c,α,ξ,np.array([1,1,1,1])))
print(p_new)

q_new = quant_logit(p_new, α, ξ)
print(q_new)

[1.7343892  1.73438126 1.62723533 1.62722111]
[0.09174339 0.0917451  0.23610328 0.23611117]
